# Analysis of `trips-info.xml`

In [1]:
import pprint
import matplotlib.pyplot as plt
import numpy as np

from paths import normal, scenarios
from parsers.trips_info import get_trips

n_trips = get_trips(f"{normal}/trips-info.xml")
scenario_data = {}

for s in scenarios:
    trips = get_trips(f"{s}/trips-info.xml")
    scenario_data[s] = trips

### Mean distance and duration comparison for all vehicles

In [2]:
def mean_comparison():
    n_distances = np.array(list(map(lambda t: t["routeLength"], n_trips)))
    n_durations = np.array(list(map(lambda t: t["duration"], n_trips)))

    for s in scenarios:
        trips = scenario_data[s]
        distances = np.array(list(map(lambda t: t["routeLength"], trips)))
        durations = np.array(list(map(lambda t: t["duration"], trips)))
        print("Distance")
        print(f"{normal}:", n_distances.mean() / 1000)
        print(f"{s}:", distances.mean() / 1000)
        print("Difference:", (distances.mean() - n_distances.mean()) / 1000, "\n")

        print("Duration")
        print(f"{normal}:", n_durations.mean() / 60)
        print(f"{s}:", durations.mean() / 60)
        print("Difference:", (durations.mean() - n_durations.mean()) / 60, "\n")

mean_comparison()

Distance
normal-output: 2.9940531725265735
apats-output: 3.5280345625511043
Difference: 0.5339813900245304 

Duration
normal-output: 6.249727446170619
apats-output: 7.514690651403653
Difference: 1.2649632052330342 



### Mean distance/duration comparison only for rerouted vehicles

In [3]:
def mean_comparison_rr():
    for s in scenarios:
        trips = scenario_data[s]

        rr_trips = list(filter(lambda t: t["rerouteNo"] > 0, trips))
        rr_distances = np.array(list(map(lambda t: t["routeLength"], rr_trips)))
        rr_durations = np.array(list(map(lambda t: t["duration"], rr_trips)))

        rr_ids = list(map(lambda t: t["id"], rr_trips))
        filtered_n_trips = list(filter(lambda t: t["id"] in rr_ids, n_trips))
        n_distances = np.array(list(map(lambda t: t["routeLength"], filtered_n_trips)))
        n_durations = np.array(list(map(lambda t: t["duration"], filtered_n_trips)))

        print("Distance")
        print(f"{normal}:", n_distances.mean() / 1000)
        print(f"{s}:", rr_distances.mean() / 1000)
        print("Difference:", (rr_distances.mean() - n_distances.mean()) / 1000, "\n")

        print("Duration")
        print(f"{normal}:", n_durations.mean() / 60)
        print(f"{s}:", rr_durations.mean() / 60)
        print("Difference:", (rr_durations.mean() - n_durations.mean()) / 60, "\n")

mean_comparison_rr()

Distance
normal-output: 3.723337022332506
apats-output: 5.343833325062035
Difference: 1.6204963027295294 

Duration
normal-output: 7.570140612076096
apats-output: 11.395781637717123
Difference: 3.8256410256410267 



### Additional distance/duration for rerouted vehicles

In [4]:
def additional_mean_comparison_rr():
    for s in scenarios:
        trips = scenario_data[s]

        rr_trips = list(filter(lambda t: t["rerouteNo"] > 0, trips))
        rr_distances = np.array(list(map(lambda t: t["routeLength"], rr_trips)))
        rr_durations = np.array(list(map(lambda t: t["duration"], rr_trips)))

        rr_ids = list(map(lambda t: t["id"], rr_trips))
        filtered_n_trips = list(filter(lambda t: t["id"] in rr_ids, n_trips))
        n_ids = list(map(lambda t: t["id"], filtered_n_trips))

        additional_distances = []
        additional_durations = []
        for t in rr_trips:
            nt = list(filter(lambda nt: t["id"] == nt["id"], filtered_n_trips))[0]
            additional_distances.append(t["routeLength"] - nt["routeLength"])
            additional_durations.append(t["duration"] - nt["duration"])

        additional_distances = np.array(additional_distances)
        additional_durations = np.array(additional_durations)
        
        print("Avg additional distance:", additional_distances.mean() / 1000, "km")
        print("Avg additional duration:", additional_durations.mean() / 60, "mins")

additional_mean_comparison_rr()

Avg additional distance: 1.6204963027295285 km
Avg additional duration: 3.825641025641026 mins
